In [20]:
# REQUIREMENTS:
# pip install reportlab qrcode pillow

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.lib import colors
from datetime import datetime
import qrcode, os, math

# ---------- Text + layout helpers ----------
def _fit(text, font_name, max_font, min_font, box_w):
    t = "" if text is None else str(text)
    s = max_font
    while s > min_font and pdfmetrics.stringWidth(t, font_name, s) > box_w:
        s -= 0.5
    return max(min_font, s)

def _draw_scaled_title(c, text, page_w, top_y, side_margin, font_bold="Helvetica-Bold",
                       max_font=18, min_font=10):
    box_w = page_w - 2*side_margin
    fs = _fit(text, font_bold, max_font, min_font, box_w)
    c.setFont(font_bold, fs)
    c.setFillColor(colors.black)
    c.drawCentredString(page_w/2, top_y, text)
    return fs

def _draw_field_row(c, top_y, height, left, usable_w, pad, cols, prefill,
                    label_fonts=("Helvetica","Helvetica-Bold"),
                    label_sizes=(10,7.2), value_font_max=11.5):
    c.setLineWidth(1.1)
    c.setStrokeColor(colors.black)
    c.rect(left, top_y-height, usable_w, height, stroke=1, fill=0)
    x=left; acc=0.0
    for i,(field,ratio) in enumerate(cols):
        acc+=ratio
        if i<len(cols)-1:
            x_next = left + usable_w*acc
            c.line(x_next, top_y-height, x_next, top_y)
        col_w = usable_w*ratio
        label = field.get("label","")
        key   = field.get("key","")
        fs = _fit(label, label_fonts[0], label_sizes[0], label_sizes[1], col_w-2*pad)
        c.setFont(label_fonts[0], fs); c.setFillColor(colors.black)
        c.drawString(x+pad, top_y - pad - fs*0.2, label)
        val = str(prefill.get(key,""))
        if val:
            vfs = _fit(val, label_fonts[1], value_font_max, 8, col_w-2*pad)
            c.setFont(label_fonts[1], vfs)
            y_val = top_y - max(6.8*mm, min(height-3.0*mm, height*0.55))
            c.drawString(x+pad, y_val, val)
        x = left + usable_w*acc

def _ensure_pretty_date(prefill):
    iso = str(prefill.get("date_iso","")).strip()
    name= str(prefill.get("date_name","")).strip()
    if iso and not name:
        try:
            dt = datetime.strptime(iso, "%Y-%m-%d")
            prefill["date_name"] = dt.strftime("%b. %d, %Y")
        except Exception:
            pass
    return prefill

def _make_qr_temp(value, box_mm=16, border=1):
    if not value: return None, 0
    path = "po_qr_temp.png"
    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=4, border=border)
    qr.add_data(value); qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(path)
    return path, box_mm*mm

def _fmt_money(v):
    try:
        return f"{float(v):,.2f}"
    except:
        return str(v)

def _fmt_qty(v):
    try:
        f = float(v)
        return f"{f:,.1f}" if abs(f - int(f)) > 1e-9 else f"{int(f):,}"
    except:
        return str(v)

# ---------- Table helpers ----------
def _draw_table_header(c, x, y_top, col_ws, row_h, headers, font_bold="Helvetica-Bold"):
    total_w = sum(col_ws)
    c.setLineWidth(0.9)
    c.setStrokeColor(colors.black)
    c.rect(x, y_top - row_h, total_w, row_h, stroke=1, fill=0)
    cx = x
    for cw in col_ws[:-1]:
        cx += cw
        c.line(cx, y_top - row_h, cx, y_top)
    c.setFont(font_bold, 8.4)
    cur_x = x
    for i,h in enumerate(headers):
        c.drawString(cur_x + 1.6*mm, y_top - row_h + 2.4*mm, str(h))
        cur_x += col_ws[i]

def _draw_table_rows(c, x, y_top, col_ws, row_h, rows):
    total_w = sum(col_ws)
    n = len(rows)
    c.setLineWidth(0.8); c.setStrokeColor(colors.black)
    c.rect(x, y_top - n*row_h, total_w, n*row_h, stroke=1, fill=0)
    cx = x
    for cw in col_ws[:-1]:
        cx += cw
        c.line(cx, y_top - n*row_h, cx, y_top)
    for r in range(1, n):
        y = y_top - r*row_h
        c.line(x, y, x + total_w, y)
    c.setFont("Helvetica", 8.0)
    for r, row in enumerate(rows):
        baseline = y_top - r*row_h - (row_h*0.68)
        cur_x = x
        for i, cw in enumerate(col_ws):
            text = "" if i >= len(row) or row[i] is None else str(row[i])
            avail = cw - 2.4*mm
            fs = _fit(text, "Helvetica", 8.0, 6.2, avail)
            c.setFont("Helvetica", fs)
            c.drawString(cur_x + 1.6*mm, baseline, text)
            cur_x += cw

def _paginate_table(c, page_w, page_h, margins_mm, start_cursor, col_mm, headers, row_h_mm, rows,
                    title_for_cont="PURCHASE ORDER – FUEL (cont.)", label_fonts=("Helvetica","Helvetica-Bold")):
    m = margins_mm*mm
    left = m
    col_ws = [wmm*mm for wmm in col_mm]
    row_h = row_h_mm*mm
    cursor = start_cursor
    i = 0
    while i < len(rows):
        min_needed = row_h * 2.0
        if (cursor - m) < min_needed:
            c.showPage()
            c.setTitle(title_for_cont)
            _draw_scaled_title(c, title_for_cont, page_w, page_h - m - 5*mm, m, label_fonts[1], 14, 9)
            cursor = page_h - m - 12*mm
        _draw_table_header(c, left, cursor, col_ws, row_h, headers, label_fonts[1])
        cursor -= row_h
        max_rows_here = max(1, int(math.floor((cursor - m) / row_h)))
        end = min(len(rows), i + max_rows_here)
        _draw_table_rows(c, left, cursor, col_ws, row_h, rows[i:end])
        cursor -= (end - i) * row_h
        i = end
    return cursor

# ---------- Main renderer ----------
def render_po_form_complete(cfg=None):
    cfg = cfg or {}
    title = cfg.get("title","PURCHASE ORDER – FUEL")
    filename = cfg.get("filename", os.path.join(os.getcwd(), "po_fuel_empty_form_a4.pdf"))
    page_size = cfg.get("page_size", A4)
    margins_mm = cfg.get("margins_mm", 6)
    pad_mm = cfg.get("pad_mm", 3)
    label_font = cfg.get("label_font", ("Helvetica","Helvetica-Bold"))
    label_font_sizes = cfg.get("label_font_sizes",(10,7.2))
    value_font_max = cfg.get("value_font_max", 11.5)
    title_font_sizes = cfg.get("title_font_sizes",(18,10))
    qr_mm = cfg.get("qr_mm", 16)
    qr_border = cfg.get("qr_border", 1)
    base_row_height_mm = cfg.get("table_row_h_mm", 8.0)
    extra_top_gap_mm = cfg.get("tables_extra_top_gap_mm", 6)

    w,h = page_size
    m = margins_mm*mm
    pad = pad_mm*mm
    left, right = m, w - m
    usable_w = right - left

    HEIGHTS = {"single":10*mm,"multi":14*mm}
    FIELDS = [
        ([({"key":"form_name","label":"Form Name"},0.26),
          ({"key":"current_form_qr_code","label":"Form QR Code"},0.20),
          ({"key":"previous_form_qr_code","label":"Previous Form QR"},0.20),
          ({"key":"type","label":"Type"},0.12),
          ({"key":"tin","label":"TIN"},0.22)],"single"),
        ([({"key":"location","label":"Location"},0.50),
          ({"key":"branch","label":"Branch"},0.50)],"single"),
        ([({"key":"date_iso","label":"Date (YYYY-MM-DD)"},0.40),
          ({"key":"date_name","label":"Date (Name e.g., Oct. 12, 2025)"},0.40),
          ({"key":"created","label":"Created (ISO)"},0.20)],"single"),
        ([({"key":"id","label":"Document ID"},0.70),
          ({"key":"generated_by","label":"Generated By"},0.30)],"single"),
    ]

    p = dict(cfg.get("prefill",{}))
    prefill = {
        "form_name": p.get("form_name", "Purchase Order – Fuel"),
        "current_form_qr_code": p.get("current_form_qr_code",""),
        "previous_form_qr_code": p.get("previous_form_qr_code",""),
        "type": p.get("type","fuel"),
        "tin": p.get("tin",""),
        "location": p.get("location",""),
        "branch": p.get("branch", p.get("branch_name","JEF Biosciences Fuel Division")),
        "date_iso": p.get("date", p.get("date_iso","")),
        "date_name": p.get("date_name",""),
        "created": p.get("created",""),
        "id": p.get("id",""),
        "generated_by": p.get("generated_by","JEF Quickeln"),
        "cashier_employee_number": "",     # <<< empty for handwriting
        "recorder_employee_number": "",    # <<< empty for handwriting
    }
    prefill = _ensure_pretty_date(prefill)

    # Empty fuel line items (handwriting space only)
    fuel_items = []  # <<< empty list
    rows = []
    total_qty = 0.0
    total_amt = 0.0
    for it in fuel_items:
        pr = it.get("price","")
        qty = it.get("quantity","")
        amt = it.get("amount","")
        try: total_qty += float(qty)
        except: pass
        try: total_amt += float(amt)
        except: pass
        rows.append([
            it.get("pump_name",""),
            it.get("po_number",""),
            it.get("plate_number",""),
            it.get("route",""),
            it.get("driver",""),
            it.get("product",""),
            _fmt_money(pr),
            _fmt_qty(qty),
            _fmt_money(amt),
        ])

    # Pre-fill with blank rows for handwriting
    min_rows = int(cfg.get("min_table_rows", 16))
    while len(rows) < min_rows:
        rows.append(["","","","","","","","",""])

    headers = ["Pump Name","PO No","Plate No","Route","Driver","Product","Price","Qty","Amount"]
    col_mm  = [28,18,25,34,22,22,14,12,22]
    row_h_mm = base_row_height_mm

    c = canvas.Canvas(filename, pagesize=page_size)
    c.setTitle(title)

    title_y = h - m - 5*mm
    _draw_scaled_title(c, title, w, title_y, m, label_font[1], title_font_sizes[0], title_font_sizes[1])
    cursor = title_y - 3*mm

    qr_value = str(prefill.get("current_form_qr_code","")).strip()
    qr_path, qr_box = _make_qr_temp(qr_value, box_mm=qr_mm, border=qr_border)
    if qr_path:
        x_qr = (w - qr_box)/2
        y_qr = cursor - qr_box - 1.6*mm
        c.drawImage(qr_path, x_qr, y_qr, width=qr_box, height=qr_box, preserveAspectRatio=True, mask='auto')
        c.setFont(label_font[0],7.2)
        c.drawCentredString(w/2, y_qr - 1.6*mm, qr_value)
        cursor = y_qr - 4.5*mm

    for cols, hspec in FIELDS:
        h_row = HEIGHTS[hspec] if isinstance(hspec, str) else float(hspec)
        if cursor - h_row < m + 24:
            c.showPage(); c.setTitle(title + " (cont.)")
            _draw_scaled_title(c, title + " (cont.)", w, h - m - 5*mm, m, label_font[1], 16, 9)
            cursor = h - m - 12*mm
        _draw_field_row(c, cursor, h_row, left, usable_w, pad, cols, prefill,
                        label_fonts=label_font, label_sizes=label_font_sizes, value_font_max=value_font_max)
        cursor -= h_row

    cursor -= (extra_top_gap_mm*mm)

    c.setFont(label_font[1], 9.2)
    c.drawString(left, cursor - 1.6*mm, f"FUEL PO LINE ITEMS ({len(rows)} rows)")
    cursor -= 5.6*mm

    if (cursor - m) < (row_h_mm*mm*2):
        c.showPage(); c.setTitle(title + " (cont.)")
        _draw_scaled_title(c, title + " (cont.)", w, h - m - 5*mm, m, label_font[1], 16, 9)
        cursor = h - m - 12*mm

    cursor = _paginate_table(
        c, w, h, margins_mm, cursor,
        col_mm, headers, row_h_mm, rows,
        title_for_cont=title + " (cont.)",
        label_fonts=label_font
    )

    # Totals strip (zeroed since empty)
    cursor -= 5*mm


    # Footer personnel fields (empty for handwriting)
    footer_cols = [
        ({"key":"cashier_employee_number","label":"Cashier Employee Number"}, 0.50),
        ({"key":"recorder_employee_number","label":"Recorder Employee Number"}, 0.50),
    ]
    h_row = 14*mm
    if cursor - h_row < m + 10:
        c.showPage(); c.setTitle(title + " (cont.)")
        _draw_scaled_title(c, title + " (cont.)", w, h - m - 5*mm, m, label_font[1], 16, 9)
        cursor = h - m - 16*mm
    _draw_field_row(c, cursor, h_row, left, usable_w, pad, footer_cols, prefill,
                    label_fonts=label_font, label_sizes=label_font_sizes, value_font_max=value_font_max)

    c.save()
    try:
        if qr_path and os.path.exists(qr_path): os.remove(qr_path)
    except: pass
    return os.path.abspath(filename)

# ===== Example usage: EMPTY form (no line items; empty cashier/recorder) =====
cfg = {
    "filename":"po_fuel_empty_form_a4.pdf",
    "prefill": {
        "id": "EMPTY-PO-20251012-000",
        "current_form_qr_code": "20251012",
        "previous_form_qr_code": "20251011",
        "type": "fuel",
        "tin": "123-456-789",
        "location": "JEF Gas Station – Sikatuna Branch",
        "branch": "JEF Biosciences Fuel Division",
        "created": "2025-10-12T14:20:00Z",
        "date": "2025-10-12",
        "fuel": [],                         # <<< no items
        "cashier_employee_number": "",      # <<< empty
        "recorder_employee_number": ""      # <<< empty
    },
    "min_table_rows": 16,
    "table_row_h_mm": 8.0,
    "qr_mm": 16
}
render_po_form_complete(cfg)


'h:\\github4\\jefstore-gasstations-backend\\reportlab\\purchase_orders\\empty\\po_fuel_empty_form_a4.pdf'